In [1]:
import $file.^.Magic

import $file.$

In [2]:
val datasetUrl = s"https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch07/01_main-chapter-code/instruction-data.json"
val outputDir = "data/instruction-data-raw"

datasetUrl: String = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch07/01_main-chapter-code/instruction-data.json"
outputDir: String = "data/instruction-data-raw"

In [3]:
Magic.!("curl", "--create-dirs", "-O", "--output-dir", outputDir, datasetUrl)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  198k  100  198k    0     0   955k      0 --:--:-- --:--:-- --:--:--  960k


In [4]:
case class InstructionDataRecord(
  instruction: String,
  input: String,
  output: String
) {
  lazy val alpacaFormat: String = {
    val formattedInput = if (input.nonEmpty) s"\n### Input:\n$input\n" else ""
    s"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
       |
       |### Instruction:
       |$instruction
       |$formattedInput
       |### Response:
       |$output
       |""".stripMargin
  }
}

defined class InstructionDataRecord

In [5]:
import $ivy.`com.lihaoyi::upickle:4.1.0`

import scala.io.Source
import upickle.default.{read, Reader, macroR}

implicit val instructionDataRecordReader: Reader[InstructionDataRecord] = macroR

val datasetRaw = Source.fromFile(s"$outputDir/instruction-data.json").mkString
type Dataset = Vector[InstructionDataRecord]
val instructionDataRecords = read[Dataset](datasetRaw)

println(s"Instruction data records count: ${instructionDataRecords.size}")

Instruction data records count: 1100


import $ivy.$
import scala.io.Source
import upickle.default.{read, Reader, macroR}
instructionDataRecordReader: Reader[InstructionDataRecord] = ammonite.$sess.cmd5$Helper$$anon$1@1a03b4c5
datasetRaw: String = """[
    {
        "instruction": "Evaluate the following phrase by transforming it into the spelling given.",
        "input": "freind --> friend",
        "output": "The spelling of the given phrase \"freind\" is incorrect, the correct spelling is \"friend\"."
    },
    {
        "instruction": "Edit the following sentence for grammar.",
        "input": "He go to the park every day.",
        "output": "He goes to the park every day."
    },
    {
        "instruction": "Convert 45 kilometers to meters.",
        "input": "",
        "output": "45 kilometers is 45000 meters."
    },
    {
        "instruction": "Rewrite this sentence to start with 'Although': Despite the rain, they went for a walk.",
        "input": "",
        "output": "Although it was raining, they went fo

In [8]:
println(instructionDataRecords.find(_.input.nonEmpty).get.alpacaFormat)
println("-----\n")
println(instructionDataRecords.find(_.input.isEmpty).get.alpacaFormat)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend

### Response:
The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".

-----

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Convert 45 kilometers to meters.

### Response:
45 kilometers is 45000 meters.



In [10]:
type Training = Dataset
type Validation = Dataset
type Test = Dataset

def split(dataset: Vector[InstructionDataRecord], trainingFraction: Double, validationFraction: Double): (Training, Validation, Test) = {
  val trainingSize = (dataset.size * trainingFraction).floor.toInt
  val validationSize = (dataset.size * validationFraction).floor.toInt

  val (training, remainingRecords) = dataset.splitAt(trainingSize)
  val (validation, test) = remainingRecords.splitAt(validationSize)
  (training, validation, test)
}

val (training, validation, test) = split(instructionDataRecords, trainingFraction = 0.85, validationFraction = 0.05) 

println(s"Training set size: ${training.size}")
println(s"Validation set size: ${validation.size}")
println(s"Test set size: ${test.size}")

Training set size: 935
Validation set size: 55
Test set size: 110


defined type Training
defined type Validation
defined type Test
defined function split
training: Training = Vector(
  InstructionDataRecord(
    instruction = "Evaluate the following phrase by transforming it into the spelling given.",
    input = "freind --> friend",
    output = "The spelling of the given phrase \"freind\" is incorrect, the correct spelling is \"friend\"."
  ),
  InstructionDataRecord(
    instruction = "Edit the following sentence for grammar.",
    input = "He go to the park every day.",
    output = "He goes to the park every day."
  ),
  InstructionDataRecord(
    instruction = "Convert 45 kilometers to meters.",
    input = "",
    output = "45 kilometers is 45000 meters."
  ),
  InstructionDataRecord(
    instruction = "Rewrite this sentence to start with 'Although': Despite the rain, they went for a walk.",
    input = "",
    output = "Although it was raining, they went for a walk."
  ),
  InstructionDataRecord(
    instruction = "What are the first 10 square